# 💣 Fabric Clear Workspace Utility

⚠️ **DESTRUCTIVE OPERATION**

This notebook deletes **all Fabric items and folders** in the current workspace,
except this notebook.

Use only in **non-production** environments for POCs or migration resets.

## What this does

- Deletes all Fabric artifacts in the workspace (except this notebook)
- Deletes folders after items are removed
- Designed for controlled workspace resets

## How to use

1. Review the configuration values in the next cell
2. Run with `dryRun = True` first
3. Set `dryRun = False` when you are confident
4. Run the execution cell at the bottom

## Documentation

The **full, safety-first implementation**, limitations, and usage guidance
are documented in the GitHub repository README:

👉 https://github.com/thedenotebook/Fabric_ClearWorkspace/blob/main/README.md

Always refer to the README before making changes or running in new environments.


## Libraries

In [ ]:
from tqdm import tqdm
import sempy.fabric as fabric
import time

## Configuration

In [ ]:
# MUST exactly match the current workspace name (case-sensitive) or the script will abort.
confirmWorkspaceName = 'Workspace_XXX'

# Dry run = True means "print what will be deleted" and DO NOT delete anything.
dryRun = False

# Optional: a short countdown before deletion begins (extra safety)
countDown = 10  # Countdown

## Functions

In [ ]:
def clear_workspace(confirm_workspace_name: str, dry_run: bool = True, count_down: int = 10):

    workspaceId = fabric.get_notebook_workspace_id()
    workspaceName = fabric.resolve_workspace_name(workspaceId)
    
    # ----------------------------
    # Constants
    # ----------------------------
    UNSUPPORTED_TYPES = {"Dashboard"}     # delete not supported via API
    DEPENDENT_TYPES   = {"SQLEndpoint"}   # deleted with parent artifact

    # ----------------------------
    # Workspace guard
    # ----------------------------
    
    if confirm_workspace_name != workspaceName:
        raise ValueError(
            f"Workspace name mismatch.\n"
            f"Provided : {confirm_workspace_name}\n"
            f"Actual   : {workspaceName}\n"
            f"Aborting."
        )

    # Helper for folder parent id nulls
    def _is_null(x):
        return x is None or (isinstance(x, str) and x.upper() == "NULL")

    # ----------------------------
    # Get Data
    # ----------------------------

    items = fabric.list_items()
    nb_id = fabric.get_artifact_id()
    items_to_delete = items.query("Id != @nb_id").copy() #ignore this workbook

    if items_to_delete.empty:
        print("\n" + "=" * 80)
        print("No deletable items found (workspace contains only this notebook).")
        print("=" * 80)
        return

    # Sort for deterministic, readable logging
    items_to_delete = items_to_delete.sort_values(["Type", "Display Name"])

    #Get Folders
    folders = fabric.list_folders()

    print("\n" + "=" * 80)
    print(f"CLEARING WORKSPACE: {workspaceName}")
    print(f"Dry run: {dry_run}")
    print(f"Items to process: {len(items_to_delete)}")
    print(f"Folders to process: {len(folders)}")
    print("=" * 80)

    
    # ----------------------------
    # Countdown (extra safety) 
    # ----------------------------
    if not dry_run and count_down > 0:
        print(f"\n🔴 WARNING: Deletion begins in {count_down} seconds.")
        for remaining in range(count_down, 0, -1):
            print(f"{remaining} seconds remaining... 🚨 abort now if unsure")
            time.sleep(1)

    # ----------------------------
    # Delete Items
    # ----------------------------
    current_type = None
    itemsuccesses = []
    itemfailures = []
    itemskipped = []

    print("\n" + "=" * 80)
    print(f"{'ITEMS LOG':^80}")
    print("=" * 80)

    for _, row in items_to_delete.iterrows():
        item_id = row["Id"]
        item_name = row.get("Display Name", "<unknown>")
        item_type = row.get("Type", "UNKNOWN")

        if item_type != current_type:
            current_type = item_type
            print("\n" + "-" * 80)
            print(f"{current_type:^80}")
            print("-" * 80)

        if item_type in UNSUPPORTED_TYPES:
            print(f" - {item_name} → SKIPPED (unsupported by API)")
            itemskipped.append(item_id)
            continue

        if item_type in DEPENDENT_TYPES:
            print(f" - {item_name} → SKIPPED (dependent artifact)")
            itemskipped.append(item_id)
            continue

        try:
            if dry_run:
                print(f" - {item_name} → WOULD DELETE")
            else:
                fabric.delete_item(item_id)
                print(f" - {item_name} → DELETED")
                itemsuccesses.append(item_id)

        except Exception as e:
            msg = str(e)
            if "404" in msg and "EntityNotFound" in msg:
                print(f" - {item_name} → ALREADY DELETED")
                itemsuccesses.append(item_id)
            else:
                print(f" - {item_name} → FAILED ({e})")
                itemfailures.append((item_id, repr(e)))

    print("\n" + "=" * 80)
    print(f"{'ITEM SUMMARY':^80}")
    print("=" * 80)
    print(f"Successful deletions: {len(itemsuccesses)}")
    print(f"Failures:             {len(itemfailures)}")
    print(f"Skipped:             {len(itemskipped)}")

    # ----------------------------
    # FOLDER DELETION
    # ----------------------------
    print("\n" + "=" * 80)
    print(f"{'FOLDERS LOG':^80}")
    print("=" * 80)

    if dry_run:
        print(" - DRY RUN: Folder deletion skipped.")
        return

    foldersuccesses = []
    folderfailures = []
    folderSkipped = []

    # Stage 1: delete non-root leaf folders
    round_num = 0
    while True:
        round_num += 1
        folders = fabric.list_folders()

        if folders is None or folders.empty:
            break

        folders = folders.copy()
        folders["Parent Folder Id"] = folders["Parent Folder Id"].apply(
            lambda x: None if _is_null(x) else x
        )

        folder_ids = set(folders["Id"])
        parent_ids = set(pid for pid in folders["Parent Folder Id"] if pid is not None)
        leaf_ids = folder_ids - parent_ids

        stage1_df = folders[
            (folders["Id"].isin(leaf_ids)) &
            (folders["Parent Folder Id"].notna())
        ].sort_values("Display Name")

        if stage1_df.empty:
            print("\n" + "-" * 80)
            print(f"{'Stage 1 complete: no non-root leaf folders left':^80}")
            print("-" * 80)
            break

        print("\n" + "-" * 80)
        print(f"{('Stage 1 - Round ' + str(round_num) + ': delete child folders'):^80}")
        print("-" * 80)

        deleted_this_round = 0
        for _, row in stage1_df.iterrows():
            try:
                fabric.delete_folder(row["Id"])
                print(f" - {row['Display Name']} → DELETED")
                foldersuccesses.append(row["Id"])
                deleted_this_round += 1
            except Exception as e:
                print(f" - {row['Display Name']} → FAILED ({e})")
                folderfailures.append((row["Id"], repr(e)))

        if deleted_this_round == 0:
            print("\nStopping to avoid infinite loop.")
            break

    # Stage 2: delete root folders if empty
    folders = fabric.list_folders()
    if folders is not None and not folders.empty:
        folders = folders.copy()
        folders["Parent Folder Id"] = folders["Parent Folder Id"].apply(
            lambda x: None if _is_null(x) else x
        )

        items_remaining = fabric.list_items()
        non_empty_folder_ids = set()
        if items_remaining is not None and "Folder Id" in items_remaining.columns:
            non_empty_folder_ids = {
                str(fid) for fid in items_remaining["Folder Id"].dropna()
                if str(fid).upper() != "NULL"
            }

        stage2_df = folders[folders["Parent Folder Id"].isna()].sort_values("Display Name")

        print("\n" + "-" * 80)
        print(f"{'Stage 2: delete root folders':^80}")
        print("-" * 80)

        for _, row in stage2_df.iterrows():
            if str(row["Id"]) in non_empty_folder_ids:
                print(f" - {row['Display Name']} → SKIPPED (FolderNotEmpty)")
                folderSkipped.append(row["Id"])
                continue

            try:
                fabric.delete_folder(row["Id"])
                print(f" - {row['Display Name']} → DELETED")
                foldersuccesses.append(row["Id"])
            except Exception as e:
                print(f" - {row['Display Name']} → FAILED ({e})")
                folderfailures.append((row["Id"], repr(e)))

    print("\n" + "=" * 80)
    print(f"{'FOLDER SUMMARY':^80}")
    print("=" * 80)
    print(f"Successful deletions: {len(foldersuccesses)}")
    print(f"Failures:             {len(folderfailures)}")
    print(f"Skipped:             {len(folderSkipped)}")

## Code Execution


In [ ]:
"""
Deletes all fabric items from a workspace. Think before you delete.
"""  

clear_workspace(confirmWorkspaceName, dryRun ,countDown)